When point cloud of both are available and there are no missing point clouds, we can use principal components to approximately align two point clouds

# Part 1: Reading point cloud in dictionary, applying known arbitrary rotation and translation to the point cloud and saving it in another dictionary

In [1]:
file_name = "Part1_STL_10_pow_4_ASCII.txt"

In [2]:
with open(file_name) as file_object:
    lines = file_object.readlines()

In [3]:
data = {}

In [4]:
i = 1
for line in lines:
    items = line.split(",")
    x, y, z = float(items[0]), float(items[1]), float(items[2])
    list_x_y_z = [x, y, z]
    data[i] = list_x_y_z
    i+=1

In [5]:
data[10029]

[0.91336119, 0.46265504, 0.37359932]

In [6]:
data[1]

[0.97175932, 0.27299145, 0.60483378]

In [7]:
len(data)

10029

Applying translation

In [8]:
transformed_data = {}

In [9]:
tx, ty, tz = 1.0, 2.0, 3.0

In [10]:
for key, value in data.items():
    transformed_data[key] = [value[0]+tx, value[1]+ty, value[2]+tz]    

In [11]:
transformed_data[1]

[1.9717593199999999, 2.27299145, 3.60483378]

In [12]:
transformed_data[10029]

[1.91336119, 2.46265504, 3.37359932]

Applying rotation of 45 degress along z-axis of coordinate frame, just to keep it simple.

In [13]:
rotation_z = [[0.717, -0.717, 0],
             [0.717, 0.717, 0],
             [0, 0, 1]]

In [14]:
for key, value in transformed_data.items():
    transformed_data[key] = [rotation_z[0][0]*value[0] + rotation_z[0][1]*value[1] + rotation_z[0][2]*value[2],
                            rotation_z[1][0]*value[0] + rotation_z[1][1]*value[1] + rotation_z[1][2]*value[2],
                            rotation_z[2][0]*value[0] + rotation_z[2][1]*value[1] + rotation_z[2][2]*value[2]]

In [15]:
transformed_data[1]

[-0.21598343721000024, 3.04348630209, 3.60483378]

In [16]:
transformed_data[10029]

[-0.39384369045, 3.1376036369099998, 3.37359932]

Original and transformed data visualization

In [17]:
import numpy as np
import ipyvolume as ipv

In [18]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)

In [19]:
for key, value in data.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

In [20]:
x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)

In [21]:
for key, value in transformed_data.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])

In [22]:
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

The red is reference point cloud and blue is transformed point cloud.

# Part 2: Calculate centroid of both point clouds

In [23]:
mean_x, mean_y, mean_z = 0.0, 0.0, 0.0
mean_x1, mean_y1, mean_z1 = 0.0, 0.0, 0.0

In [24]:
for key, value in data.items():
    mean_x += value[0]
    mean_y += value[1]
    mean_z += value[2]
    
mean_x = mean_x / len(data)
mean_y = mean_y / len(data)
mean_z = mean_z / len(data)

In [25]:
mean_x

0.677856793295442

In [26]:
for key, value in transformed_data.items():
    mean_x1 += value[0]
    mean_y1 += value[1]
    mean_z1 += value[2]
    
mean_x1 = mean_x1 / len(data)
mean_y1 = mean_y1 / len(data)
mean_z1 = mean_z1 / len(data)

In [27]:
mean_x1

-0.383403964464602

# Part 3: Compute eigen values and eigen vectors of both the point clouds using covariance matrix of point clouds

calculate covariance matrix of source

In [28]:
cov_matrix1 = np.zeros((3, 3))
for i in range(1, 10030):
    cov_matrix1[0][0] += (transformed_data[i][0] - mean_x1)**2
    cov_matrix1[1][1] += (transformed_data[i][1] - mean_y1)**2
    cov_matrix1[2][2] += (transformed_data[i][2] - mean_z1)**2
    cov_matrix1[1][0] += (transformed_data[i][1] - mean_y1)*(transformed_data[i][0] - mean_x1)
    cov_matrix1[2][0] += (transformed_data[i][2] - mean_z1)*(transformed_data[i][0] - mean_x1)
    cov_matrix1[2][1] += (transformed_data[i][2] - mean_z1)*(transformed_data[i][1] - mean_y1)
cov_matrix1[0][1] = cov_matrix1[1][0]
cov_matrix1[0][2] = cov_matrix1[2][0]
cov_matrix1[1][2] = cov_matrix1[2][1]
cov_matrix1

array([[  70.30006934,  154.4618088 ,   14.70613606],
       [ 154.4618088 ,  864.19996067, -120.02468181],
       [  14.70613606, -120.02468181,   85.37460139]])

In [29]:
eigenValue1, eigenVector1 = np.linalg.eig(cov_matrix1)

In [30]:
eigenValue1, eigenVector1

(array([909.34419802,  16.4166284 ,  94.11380499]),
 array([[-0.17694516, -0.79687337,  0.57765323],
        [-0.97438762,  0.22458847,  0.01134833],
        [ 0.13877743,  0.56085012,  0.81620339]]))

Sort eigen values and eigen vectors

In [31]:
idx = np.argsort(eigenValue1)
eigenValue1_sorted = eigenValue1[idx]
eigenVector1_sorted = eigenVector1[:,idx]
eigenValue1_sorted, eigenVector1_sorted

(array([ 16.4166284 ,  94.11380499, 909.34419802]),
 array([[-0.79687337,  0.57765323, -0.17694516],
        [ 0.22458847,  0.01134833, -0.97438762],
        [ 0.56085012,  0.81620339,  0.13877743]]))

In [32]:
cov_matrix = np.zeros((3, 3))
for i in range(1, 10030):
    cov_matrix[0][0] += (data[i][0] - mean_x)**2
    cov_matrix[1][1] += (data[i][1] - mean_y)**2
    cov_matrix[2][2] += (data[i][2] - mean_z)**2
    cov_matrix[1][0] += (data[i][1] - mean_y)*(data[i][0] - mean_x)
    cov_matrix[2][0] += (data[i][2] - mean_z)*(data[i][0] - mean_x)
    cov_matrix[2][1] += (data[i][2] - mean_z)*(data[i][1] - mean_y)
cov_matrix[0][1] = cov_matrix[1][0]
cov_matrix[0][2] = cov_matrix[2][0]
cov_matrix[1][2] = cov_matrix[2][1]
cov_matrix

array([[604.67333847, 386.07123053, -73.44389522],
       [386.07123053, 304.21600755, -93.95454524],
       [-73.44389522, -93.95454524,  85.37460139]])

In [33]:
eigenValue, eigenVector = np.linalg.eig(cov_matrix)
eigenValue, eigenVector

(array([884.90401212,  93.2549233 ,  16.10501199]),
 array([[-0.81375597, -0.41443035, -0.40749074],
        [-0.56384187,  0.39281401,  0.72648434],
        [ 0.14100909, -0.82094131,  0.55332793]]))

In [34]:
idx = np.argsort(eigenValue)
eigenValue_sorted = eigenValue[idx]
eigenVector_sorted = eigenVector[:,idx]
eigenValue_sorted, eigenVector_sorted

(array([ 16.10501199,  93.2549233 , 884.90401212]),
 array([[-0.40749074, -0.41443035, -0.81375597],
        [ 0.72648434,  0.39281401, -0.56384187],
        [ 0.55332793, -0.82094131,  0.14100909]]))

# Part 4: Initial alignment and visualization

4 cases possible

case 1: eigenVector1_1 = +, eigenVector1_2 = +

In [35]:
eigenVector1_sorted

array([[-0.79687337,  0.57765323, -0.17694516],
       [ 0.22458847,  0.01134833, -0.97438762],
       [ 0.56085012,  0.81620339,  0.13877743]])

In [36]:
transformed_data1 = transformed_data.copy()

In [37]:
eigenVector_sorted

array([[-0.40749074, -0.41443035, -0.81375597],
       [ 0.72648434,  0.39281401, -0.56384187],
       [ 0.55332793, -0.82094131,  0.14100909]])

In [38]:
rotation1 = np.matmul(eigenVector_sorted, np.transpose(eigenVector1_sorted))
rotation1

array([[ 0.22931167,  0.69669293, -0.67973165],
       [-0.25223666,  0.71701833,  0.64981642],
       [-0.94010256, -0.02244275, -0.34015217]])

Align source and target

In [39]:
for key, value in transformed_data1.items():
    transformed_data1[key] = [value[0]+mean_x-mean_x1, value[1]+mean_y-mean_y1, value[2]+mean_z-mean_z1]

Align axes

In [40]:
for key, value in transformed_data1.items():
    value = [rotation1[0][0]*(value[0]-mean_x)+rotation1[0][1]*(value[1]-mean_y)+rotation1[0][2]*(value[2]-mean_z),
             rotation1[1][0]*(value[0]-mean_x)+rotation1[1][1]*(value[1]-mean_y)+rotation1[1][2]*(value[2]-mean_z),
             rotation1[2][0]*(value[0]-mean_x)+rotation1[2][1]*(value[1]-mean_y)+rotation1[2][2]*(value[2]-mean_z)]
    
for key, value in transformed_data1.items():
    value = [value[0]+mean_x, value[1]+mean_y, value[2]+mean_z]

In [41]:
transformed_data1

{1: [0.8452773205500438, 0.466626052047308, 0.6048337799999834],
 2: [0.8502095277000438, 0.5140864402173082, 0.6004264399999832],
 3: [0.8498525047200439, 0.4888115449173078, 0.6012162599999837],
 4: [0.8360490860100441, 0.4069559927673083, 0.6124521499999838],
 5: [0.8896194470400441, 0.687864798117308, 0.5696388499999832],
 6: [0.8922809868900442, 0.6947321309073078, 0.5676646199999831],
 7: [0.8655824522400443, 0.5508054155373086, 0.5890865899999835],
 8: [0.8726633434800443, 0.5910776277573082, 0.5833598399999835],
 9: [0.8751988132200438, 0.6127975223373077, 0.5811518399999831],
 10: [0.653847914010044, -0.20531250555269187, 0.7506691799999836],
 11: [0.7629854165400441, 0.11947112153730854, 0.6687825899999837],
 12: [0.773678137920044, 0.12848695199730775, 0.6612513699999831],
 13: [0.7304254499100441, 0.11841325256730828, 0.6911463699999838],
 14: [0.7338038320200438, 0.12875440733730814, 0.6886054299999835],
 15: [0.7356946112100441, 0.12690141404730815, 0.6873480099999836],
 

In [42]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)
for key, value in data.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)
for key, value in transformed_data1.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])
        
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

wrong alignment

case 2 eigenVector1_1 = +, eigenVector1_2 = - will imply eigenVector1_3 = -

In [43]:
eigenVector1_copy = np.copy(eigenVector1_sorted)
eigenVector1_copy

array([[-0.79687337,  0.57765323, -0.17694516],
       [ 0.22458847,  0.01134833, -0.97438762],
       [ 0.56085012,  0.81620339,  0.13877743]])

In [44]:
for i in range(0,3):
    eigenVector1_copy[i][1] = -eigenVector1_copy[i][1]
    eigenVector1_copy[i][2] = -eigenVector1_copy[i][2]
    
eigenVector1_copy

array([[-0.79687337, -0.57765323,  0.17694516],
       [ 0.22458847, -0.01134833,  0.97438762],
       [ 0.56085012, -0.81620339, -0.13877743]])

In [45]:
rotation2 = np.matmul(eigenVector_sorted, np.transpose(eigenVector1_copy))
rotation2

array([[ 0.42012537, -0.87972837,  0.22264919],
       [-0.90559539, -0.39069832,  0.16508124],
       [ 0.05823798,  0.27098489,  0.96082024]])

In [46]:
transformed_data2 = transformed_data.copy()
transformed_data2

{1: [-0.21598343721000024, 3.04348630209, 3.60483378],
 2: [-0.21105123006000026, 3.09094669026, 3.60042644],
 3: [-0.2114082530400001, 3.0656717949599996, 3.60121626],
 4: [-0.2252116717499999, 2.98381624281, 3.61245215],
 5: [-0.17164131071999988, 3.26472504816, 3.56963885],
 6: [-0.16897977086999982, 3.2715923809499996, 3.56766462],
 7: [-0.19567830551999976, 3.1276656655800004, 3.58908659],
 8: [-0.18859741427999976, 3.1679378778, 3.58335984],
 9: [-0.18606194454000025, 3.1896577723799995, 3.58115184],
 10: [-0.40741284375, 2.37154774449, 3.75066918],
 11: [-0.2982753412199999, 2.6963313715800004, 3.66878259],
 12: [-0.28758261984, 2.7053472020399996, 3.66125137],
 13: [-0.3308353078499999, 2.69527350261, 3.6911463700000002],
 14: [-0.3274569257400002, 2.70561465738, 3.68860543],
 15: [-0.32556614654999994, 2.70376166409, 3.68734801],
 16: [-0.31834776339000004, 2.7045776674499997, 3.68237752],
 17: [-0.32183892243, 2.72010821727, 3.68443817],
 18: [-0.30325842669, 2.72813497490999

In [47]:
for key, value in transformed_data2.items():
    transformed_data2[key] = [value[0]+mean_x-mean_x1, value[1]+mean_y-mean_y1, value[2]+mean_z-mean_z1]
    
transformed_data2

{1: [0.8452773205500438, 0.466626052047308, 0.6048337799999834],
 2: [0.8502095277000438, 0.5140864402173082, 0.6004264399999832],
 3: [0.8498525047200439, 0.4888115449173078, 0.6012162599999837],
 4: [0.8360490860100441, 0.4069559927673083, 0.6124521499999838],
 5: [0.8896194470400441, 0.687864798117308, 0.5696388499999832],
 6: [0.8922809868900442, 0.6947321309073078, 0.5676646199999831],
 7: [0.8655824522400443, 0.5508054155373086, 0.5890865899999835],
 8: [0.8726633434800443, 0.5910776277573082, 0.5833598399999835],
 9: [0.8751988132200438, 0.6127975223373077, 0.5811518399999831],
 10: [0.653847914010044, -0.20531250555269187, 0.7506691799999836],
 11: [0.7629854165400441, 0.11947112153730854, 0.6687825899999837],
 12: [0.773678137920044, 0.12848695199730775, 0.6612513699999831],
 13: [0.7304254499100441, 0.11841325256730828, 0.6911463699999838],
 14: [0.7338038320200438, 0.12875440733730814, 0.6886054299999835],
 15: [0.7356946112100441, 0.12690141404730815, 0.6873480099999836],
 

In [48]:
for key, value in transformed_data2.items():
    transformed_data2[key] = [rotation2[0][0]*(value[0]-mean_x)+rotation2[0][1]*(value[1]-mean_y)+rotation2[0][2]*(value[2]-mean_z),
                              rotation2[1][0]*(value[0]-mean_x)+rotation2[1][1]*(value[1]-mean_y)+rotation2[1][2]*(value[2]-mean_z),
                              rotation2[2][0]*(value[0]-mean_x)+rotation2[2][1]*(value[1]-mean_y)+rotation2[2][2]*(value[2]-mean_z)]
    
for key, value in transformed_data2.items():
    transformed_data2[key] = [value[0]+mean_x, value[1]+mean_y, value[2]+mean_z]

In [49]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)
for key, value in data.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)
for key, value in transformed_data2.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])
        
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

Almost aligned

case 3 eigenVector1_1 = -, eigenVector1_2 = + will imply eigenVector1_3 = -

In [50]:
eigenVector1_copy = np.copy(eigenVector1_sorted)
eigenVector1_copy

array([[-0.79687337,  0.57765323, -0.17694516],
       [ 0.22458847,  0.01134833, -0.97438762],
       [ 0.56085012,  0.81620339,  0.13877743]])

In [51]:
for i in range(0,3):
    eigenVector1_copy[i][0] = -eigenVector1_copy[i][0]
    eigenVector1_copy[i][2] = -eigenVector1_copy[i][2]
    
eigenVector1_copy

array([[ 0.79687337,  0.57765323,  0.17694516],
       [-0.22458847,  0.01134833,  0.97438762],
       [-0.56085012,  0.81620339, -0.13877743]])

In [52]:
rotation3 = np.matmul(eigenVector, np.transpose(eigenVector1_copy))
rotation3

array([[-0.959961  , -0.21899682,  0.17468619],
       [-0.09385241,  0.83896751,  0.53602727],
       [-0.26394431,  0.49817056, -0.82592947]])

In [53]:
transformed_data3 = transformed_data.copy()
transformed_data3

{1: [-0.21598343721000024, 3.04348630209, 3.60483378],
 2: [-0.21105123006000026, 3.09094669026, 3.60042644],
 3: [-0.2114082530400001, 3.0656717949599996, 3.60121626],
 4: [-0.2252116717499999, 2.98381624281, 3.61245215],
 5: [-0.17164131071999988, 3.26472504816, 3.56963885],
 6: [-0.16897977086999982, 3.2715923809499996, 3.56766462],
 7: [-0.19567830551999976, 3.1276656655800004, 3.58908659],
 8: [-0.18859741427999976, 3.1679378778, 3.58335984],
 9: [-0.18606194454000025, 3.1896577723799995, 3.58115184],
 10: [-0.40741284375, 2.37154774449, 3.75066918],
 11: [-0.2982753412199999, 2.6963313715800004, 3.66878259],
 12: [-0.28758261984, 2.7053472020399996, 3.66125137],
 13: [-0.3308353078499999, 2.69527350261, 3.6911463700000002],
 14: [-0.3274569257400002, 2.70561465738, 3.68860543],
 15: [-0.32556614654999994, 2.70376166409, 3.68734801],
 16: [-0.31834776339000004, 2.7045776674499997, 3.68237752],
 17: [-0.32183892243, 2.72010821727, 3.68443817],
 18: [-0.30325842669, 2.72813497490999

In [54]:
for key, value in transformed_data3.items():
    transformed_data3[key] = [value[0]+mean_x-mean_x1, value[1]+mean_y-mean_y1, value[2]+mean_z-mean_z1]
    
transformed_data3

{1: [0.8452773205500438, 0.466626052047308, 0.6048337799999834],
 2: [0.8502095277000438, 0.5140864402173082, 0.6004264399999832],
 3: [0.8498525047200439, 0.4888115449173078, 0.6012162599999837],
 4: [0.8360490860100441, 0.4069559927673083, 0.6124521499999838],
 5: [0.8896194470400441, 0.687864798117308, 0.5696388499999832],
 6: [0.8922809868900442, 0.6947321309073078, 0.5676646199999831],
 7: [0.8655824522400443, 0.5508054155373086, 0.5890865899999835],
 8: [0.8726633434800443, 0.5910776277573082, 0.5833598399999835],
 9: [0.8751988132200438, 0.6127975223373077, 0.5811518399999831],
 10: [0.653847914010044, -0.20531250555269187, 0.7506691799999836],
 11: [0.7629854165400441, 0.11947112153730854, 0.6687825899999837],
 12: [0.773678137920044, 0.12848695199730775, 0.6612513699999831],
 13: [0.7304254499100441, 0.11841325256730828, 0.6911463699999838],
 14: [0.7338038320200438, 0.12875440733730814, 0.6886054299999835],
 15: [0.7356946112100441, 0.12690141404730815, 0.6873480099999836],
 

In [55]:
for key, value in transformed_data3.items():
    transformed_data3[key] = [rotation3[0][0]*(value[0]-mean_x)+rotation3[0][1]*(value[1]-mean_y)+rotation3[0][2]*(value[2]-mean_z),
                              rotation3[1][0]*(value[0]-mean_x)+rotation3[1][1]*(value[1]-mean_y)+rotation3[1][2]*(value[2]-mean_z),
                              rotation3[2][0]*(value[0]-mean_x)+rotation3[2][1]*(value[1]-mean_y)+rotation3[2][2]*(value[2]-mean_z)]
    
for key, value in transformed_data3.items():
    transformed_data3[key] = [value[0]+mean_x, value[1]+mean_y, value[2]+mean_z]

In [56]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)
for key, value in data.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)
for key, value in transformed_data3.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])
        
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

wrong alignment

case 4 eigenVector1_1 = -, eigenVector1_2 = - will imply eigenVector1_3 = +

In [57]:
eigenVector1_copy = np.copy(eigenVector1_sorted)
eigenVector1_copy

array([[-0.79687337,  0.57765323, -0.17694516],
       [ 0.22458847,  0.01134833, -0.97438762],
       [ 0.56085012,  0.81620339,  0.13877743]])

In [58]:
for i in range(0,3):
    eigenVector1_copy[i][0] = -eigenVector1_copy[i][0]
    eigenVector1_copy[i][1] = -eigenVector1_copy[i][1]
    
eigenVector1_copy

array([[ 0.79687337, -0.57765323, -0.17694516],
       [-0.22458847, -0.01134833, -0.97438762],
       [-0.56085012, -0.81620339,  0.13877743]])

In [59]:
rotation4 = np.matmul(eigenVector, np.transpose(eigenVector1_copy))
rotation4

array([[-0.33695992,  0.58451723,  0.73810407],
       [-0.80476874, -0.58570275,  0.09643429],
       [ 0.48867709, -0.56150859,  0.66775954]])

In [60]:
transformed_data4 = transformed_data.copy()
transformed_data4

{1: [-0.21598343721000024, 3.04348630209, 3.60483378],
 2: [-0.21105123006000026, 3.09094669026, 3.60042644],
 3: [-0.2114082530400001, 3.0656717949599996, 3.60121626],
 4: [-0.2252116717499999, 2.98381624281, 3.61245215],
 5: [-0.17164131071999988, 3.26472504816, 3.56963885],
 6: [-0.16897977086999982, 3.2715923809499996, 3.56766462],
 7: [-0.19567830551999976, 3.1276656655800004, 3.58908659],
 8: [-0.18859741427999976, 3.1679378778, 3.58335984],
 9: [-0.18606194454000025, 3.1896577723799995, 3.58115184],
 10: [-0.40741284375, 2.37154774449, 3.75066918],
 11: [-0.2982753412199999, 2.6963313715800004, 3.66878259],
 12: [-0.28758261984, 2.7053472020399996, 3.66125137],
 13: [-0.3308353078499999, 2.69527350261, 3.6911463700000002],
 14: [-0.3274569257400002, 2.70561465738, 3.68860543],
 15: [-0.32556614654999994, 2.70376166409, 3.68734801],
 16: [-0.31834776339000004, 2.7045776674499997, 3.68237752],
 17: [-0.32183892243, 2.72010821727, 3.68443817],
 18: [-0.30325842669, 2.72813497490999

In [61]:
for key, value in transformed_data4.items():
    transformed_data4[key] = [value[0]+mean_x-mean_x1, value[1]+mean_y-mean_y1, value[2]+mean_z-mean_z1]
    
transformed_data4

{1: [0.8452773205500438, 0.466626052047308, 0.6048337799999834],
 2: [0.8502095277000438, 0.5140864402173082, 0.6004264399999832],
 3: [0.8498525047200439, 0.4888115449173078, 0.6012162599999837],
 4: [0.8360490860100441, 0.4069559927673083, 0.6124521499999838],
 5: [0.8896194470400441, 0.687864798117308, 0.5696388499999832],
 6: [0.8922809868900442, 0.6947321309073078, 0.5676646199999831],
 7: [0.8655824522400443, 0.5508054155373086, 0.5890865899999835],
 8: [0.8726633434800443, 0.5910776277573082, 0.5833598399999835],
 9: [0.8751988132200438, 0.6127975223373077, 0.5811518399999831],
 10: [0.653847914010044, -0.20531250555269187, 0.7506691799999836],
 11: [0.7629854165400441, 0.11947112153730854, 0.6687825899999837],
 12: [0.773678137920044, 0.12848695199730775, 0.6612513699999831],
 13: [0.7304254499100441, 0.11841325256730828, 0.6911463699999838],
 14: [0.7338038320200438, 0.12875440733730814, 0.6886054299999835],
 15: [0.7356946112100441, 0.12690141404730815, 0.6873480099999836],
 

In [62]:
for key, value in transformed_data4.items():
    transformed_data4[key] = [rotation4[0][0]*(value[0]-mean_x)+rotation4[0][1]*(value[1]-mean_y)+rotation4[0][2]*(value[2]-mean_z),
                              rotation4[1][0]*(value[0]-mean_x)+rotation4[1][1]*(value[1]-mean_y)+rotation4[1][2]*(value[2]-mean_z),
                              rotation4[2][0]*(value[0]-mean_x)+rotation4[2][1]*(value[1]-mean_y)+rotation4[2][2]*(value[2]-mean_z)]
    
for key, value in transformed_data4.items():
    transformed_data4[key] = [value[0]+mean_x, value[1]+mean_y, value[2]+mean_z]

In [63]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)
for key, value in data.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)
for key, value in transformed_data4.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])
        
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

wrong alignment

# Part 5: Selection of best alignment for further iterative svd based on minimum angle of rotation

In [64]:
trace1 = rotation1[0][0] + rotation1[1][1] + rotation1[2][2]
theta1 = np.arccos((trace1-1)/2)
theta1

1.7690026441356324

In [65]:
trace2 = rotation2[0][0] + rotation2[1][1] + rotation2[2][2]
theta2 = np.arccos((trace2-1)/2)
theta2

1.5756727019593153

In [66]:
trace3 = rotation3[0][0] + rotation3[1][1] + rotation3[2][2]
theta3 = np.arccos((trace3-1)/2)
theta3

2.9106955402204413

In [67]:
trace4 = rotation4[0][0] + rotation4[1][1] + rotation4[2][2]
theta4 = np.arccos((trace4-1)/2)
theta4

2.249072333845438

The minimum angle is in rotation 2 and we got the best initial alignment with the help of rotation 2.
Another way to check select the best initial alignment is to select the rotation based on mean square error.

# SVD based ICP

Moving forward with transformed_data2

In [68]:
num_iterations = 0
iter_error = {}

In [69]:
def compute_centroid(point_cloud): # point_cloud is a dictionary
    no_points = len(point_cloud)
    centroid = [0.0, 0.0, 0.0]
    for key, value in point_cloud.items():
        for i in range(0,3):
            centroid[i] += value[i]
            
    for i in range(0,3):
        centroid[i] = centroid[i]/no_points
        
    return centroid

In [70]:
target = data.copy()
centroid_t = compute_centroid(target)
for key, value in target.items():
    for i in range(0,3):
        target[key][i] -=centroid_t[i]
        
target

{1: [0.29390252670455796, 0.060401093992421856, 0.06265161882341219],
 2: [0.3304385067045579, 0.09005812399242183, 0.05824427882341221],
 3: [0.31256408670455793, 0.07268164399242186, 0.05903409882341215],
 4: [0.245856296704558, 0.025225483992421832, 0.07026998882341218],
 5: [0.47910536670455794, 0.1837599639924218, 0.027456688823412234],
 6: [0.485750326704558, 0.18669287399242182, 0.025482458823412157],
 7: [0.36676479670455797, 0.10494379399242185, 0.04690442882341217],
 8: [0.39978648670455796, 0.1280897639924218, 0.04117767882341217],
 9: [0.41670096670455803, 0.14146802399242184, 0.03896967882341218],
 10: [-0.308167183295442, -0.27468199600757814, 0.20848701882341225],
 11: [-0.005572253295441976, -0.12430115600757817, 0.12660042882341216],
 12: [0.008171506704558018, -0.12547053600757818, 0.11906920882341221],
 13: [-0.02901565329544198, -0.10233316600757816, 0.14896420882341221],
 14: [-0.019448333295442044, -0.09747767600757817, 0.14642326882341217],
 15: [-0.0194219832954

In [71]:
source = transformed_data2.copy()
centroid_s = compute_centroid(source)
for key, value in source.items():
    for i in range(0,3):
        source[key][i] -=centroid_s[i]
        
source

{1: [-0.1391954658525042, -0.24052397052316402, 0.1387870132241168],
 2: [-0.17985686120175337, -0.26426081765826825, 0.1477006418247464],
 3: [-0.15759596032264722, -0.2539322556872929, 0.14158960974096846],
 4: [-0.08888281329748127, -0.20759628199561217, 0.1293997788193041],
 5: [-0.3230323380213365, -0.3729276248688677, 0.16750576648866267],
 6: [-0.3283951058035341, -0.37834686680012963, 0.16762483251791183],
 7: [-0.20822583833326092, -0.294400505676548, 0.14765066024214257],
 8: [-0.24195464021106505, -0.31749259277794917, 0.15347382094720952],
 9: [-0.2604886419586861, -0.3286391281860902, 0.15738575384323195],
 10: [0.4039737303770553, 0.21943357134566135, 0.08567495511763112],
 11: [0.14587180991559956, -0.019811204813293487, 0.10136406729169023],
 12: [0.14075579160396745, -0.03426021691369979, 0.09719379511357973],
 13: [0.13810245657642362, 0.013780098900380594, 0.12066874633847957],
 14: [0.12985865513795503, 0.006260918323373166, 0.12122640666100104],
 15: [0.13200318668

data visualization

In [72]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)
for key, value in target.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)
for key, value in source.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])
        
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

In [73]:
cross_cov_matrix = np.zeros((3,3))
cross_cov_matrix

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [74]:
key_list = []
for i in range(1, 10030):
    key_list.append(i)

key_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [75]:
corresponding_point_pair = []

In [76]:
import math

In [77]:
def corresponding_point(key):
    dict1 = {}
    for i in key_list:
        if i == key:
            continue
            
        euclidean_distance = math.sqrt((source[key][0] - target[i][0])**2 + (source[key][1] - target[i][1])**2 + (source[key][2] - target[i][2])**2)
        dict1[i] = euclidean_distance
        
    for key1 in dict1.keys():
        min_key = key1
        break
    for key1 in dict1.keys():
        if dict1[key1] < dict1[min_key]:
            min_key = key1
    corresponding_point_pair.append([key, min_key])
    key_list.remove(min_key)

In [78]:
for i in range(1, 10030):
    corresponding_point(i)

In [79]:
corresponding_point_pair

[[1, 3580],
 [2, 104],
 [3, 106],
 [4, 103],
 [5, 162],
 [6, 1313],
 [7, 173],
 [8, 7259],
 [9, 7306],
 [10, 589],
 [11, 281],
 [12, 279],
 [13, 292],
 [14, 278],
 [15, 283],
 [16, 287],
 [17, 282],
 [18, 218],
 [19, 569],
 [20, 285],
 [21, 286],
 [22, 291],
 [23, 289],
 [24, 214],
 [25, 215],
 [26, 290],
 [27, 288],
 [28, 556],
 [29, 560],
 [30, 550],
 [31, 606],
 [32, 558],
 [33, 559],
 [34, 557],
 [35, 272],
 [36, 273],
 [37, 270],
 [38, 304],
 [39, 302],
 [40, 306],
 [41, 294],
 [42, 553],
 [43, 552],
 [44, 608],
 [45, 549],
 [46, 297],
 [47, 318],
 [48, 311],
 [49, 305],
 [50, 309],
 [51, 308],
 [52, 315],
 [53, 316],
 [54, 307],
 [55, 327],
 [56, 312],
 [57, 303],
 [58, 205],
 [59, 368],
 [60, 367],
 [61, 208],
 [62, 276],
 [63, 360],
 [64, 352],
 [65, 9747],
 [66, 324],
 [67, 353],
 [68, 326],
 [69, 328],
 [70, 325],
 [71, 331],
 [72, 329],
 [73, 333],
 [74, 310],
 [75, 330],
 [76, 332],
 [77, 9786],
 [78, 335],
 [79, 597],
 [80, 6109],
 [81, 602],
 [82, 359],
 [83, 547],
 [84, 

In [80]:
def rmsd(point_pair):
    se = 0.0
    for pair in point_pair:
        for i in range(0,3):
            se += (source[pair[0]][i] - target[pair[1]][i])**2
    
    mse = se/10029
    rmse = math.sqrt(mse)
    
    iter_error[num_iterations] = rmse  

In [81]:
rmsd(corresponding_point_pair)
iter_error

{0: 0.14201372894228723}

1st iteration

In [82]:
cross_cov_matrix = np.zeros((3,3))
cross_cov_matrix

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [83]:
for pair in corresponding_point_pair:
    cross_cov_matrix[0][0] += source[pair[0]][0]*target[pair[1]][0]
    cross_cov_matrix[1][1] += source[pair[0]][1]*target[pair[1]][1]
    cross_cov_matrix[2][2] += source[pair[0]][2]*target[pair[1]][2]
    cross_cov_matrix[0][1] += source[pair[0]][0]*target[pair[1]][1]
    cross_cov_matrix[0][2] += source[pair[0]][0]*target[pair[1]][2]
    cross_cov_matrix[1][2] += source[pair[0]][1]*target[pair[1]][2]
    
cross_cov_matrix[1][0] = cross_cov_matrix[0][1]
cross_cov_matrix[2][0] = cross_cov_matrix[0][2]
cross_cov_matrix[2][1] = cross_cov_matrix[1][2]

cross_cov_matrix

array([[550.06254532, 344.81734294, -64.38343398],
       [344.81734294, 273.70514917, -88.39435903],
       [-64.38343398, -88.39435903,  82.16966434]])

In [84]:
u, s, vh = np.linalg.svd(cross_cov_matrix)
u, s, vh

(array([[-0.81585052,  0.42078026, -0.39665087],
        [-0.56041755, -0.40625219,  0.72172802],
        [ 0.14254862,  0.81111228,  0.5672537 ]]),
 array([798.17106011,  93.04254137,  14.72375736]),
 array([[-0.81585052, -0.56041755,  0.14254862],
        [ 0.42078026, -0.40625219,  0.81111228],
        [-0.39665087,  0.72172802,  0.5672537 ]]))

In [85]:
rotation_matrix = np.matmul(np.transpose(vh), np.transpose(u))
rotation_matrix

array([[ 1.00000000e+00, -5.55111512e-17, -1.38777878e-16],
       [-5.55111512e-17,  1.00000000e+00, -5.55111512e-17],
       [ 1.11022302e-16, -1.11022302e-16,  1.00000000e+00]])

rotate source

In [86]:
for key, value in source.items():
    source[key] = [rotation_matrix[0][0]*value[0]+rotation_matrix[0][1]*value[1]+rotation_matrix[0][2]*value[2],
                   rotation_matrix[1][0]*value[0]+rotation_matrix[1][1]*value[1]+rotation_matrix[1][2]*value[2],
                   rotation_matrix[2][0]*value[0]+rotation_matrix[2][1]*value[1]+rotation_matrix[2][2]*value[2]]
    
#for key, value in source.items():
#    source[key] = [value[0]+centroid_t[0], value[1]+centroid_t[1], value[2]+centroid_t[2]]

Data visulaization

In [87]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)
for key, value in target.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)
for key, value in source.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])
        
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

In [88]:
key_list = []
num_iterations = 1
for i in range(1, 10030):
    key_list.append(i)
corresponding_point_pair = []
for i in range(1, 10030):
    corresponding_point(i)
rmsd(corresponding_point_pair)
iter_error   

{0: 0.14201372894228723, 1: 0.1420137289422872}

In [89]:
corresponding_point_pair

[[1, 3580],
 [2, 104],
 [3, 106],
 [4, 103],
 [5, 162],
 [6, 1313],
 [7, 173],
 [8, 7259],
 [9, 7306],
 [10, 589],
 [11, 281],
 [12, 279],
 [13, 292],
 [14, 278],
 [15, 283],
 [16, 287],
 [17, 282],
 [18, 218],
 [19, 569],
 [20, 285],
 [21, 286],
 [22, 291],
 [23, 289],
 [24, 214],
 [25, 215],
 [26, 290],
 [27, 288],
 [28, 556],
 [29, 560],
 [30, 550],
 [31, 606],
 [32, 558],
 [33, 559],
 [34, 557],
 [35, 272],
 [36, 273],
 [37, 270],
 [38, 304],
 [39, 302],
 [40, 306],
 [41, 294],
 [42, 553],
 [43, 552],
 [44, 608],
 [45, 549],
 [46, 297],
 [47, 318],
 [48, 311],
 [49, 305],
 [50, 309],
 [51, 308],
 [52, 315],
 [53, 316],
 [54, 307],
 [55, 327],
 [56, 312],
 [57, 303],
 [58, 205],
 [59, 368],
 [60, 367],
 [61, 208],
 [62, 276],
 [63, 360],
 [64, 352],
 [65, 9747],
 [66, 324],
 [67, 353],
 [68, 326],
 [69, 328],
 [70, 325],
 [71, 331],
 [72, 329],
 [73, 333],
 [74, 310],
 [75, 330],
 [76, 332],
 [77, 9786],
 [78, 335],
 [79, 597],
 [80, 6109],
 [81, 602],
 [82, 359],
 [83, 547],
 [84, 

The rotation matrix approaches identity but rmsd is still large.
Need to find a way to reduce error.